In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader('./杂交水稻高产高效实用栽培技术.pdf')
splitter = RecursiveCharacterTextSplitter()
chunks = splitter.split_documents(loader.lazy_load())

In [7]:
import pandas as pd
df_chunks = pd.DataFrame({
  'text': [chunk.page_content for chunk in chunks],
  'source': [chunk.metadata['source'] for chunk in chunks],
  'page': [chunk.metadata['page'] for chunk in chunks]
})
df_chunks.to_csv('chunks.csv', sep="|", errors='ignore')

In [8]:
df_chunks = pd.read_csv('chunks.csv', sep="|", index_col=0)
df_chunks

,text,source,page
0,杂交水稻高产高效实用栽培技术\n邹应斌主编\n黄敏万克江副主编\n中国农业出版社\n北京,./杂交水稻高产高效实用栽培技术.pdf,0
1,目录\n前言\n第一章杂交水稻生育期与温光反应特性……………………………1\n第一节杂交水稻...,./杂交水稻高产高效实用栽培技术.pdf,1
2,杂交水稻高产高效实用栽培技术\n第三节杂交水稻茎秆的形态建成及株高的变化⋯⋯⋯⋯⋯⋯22\n...,./杂交水稻高产高效实用栽培技术.pdf,2
3,目录\n二、源、库的协调……………………………………………………45\n三、源、库、流的应用...,./杂交水稻高产高效实用栽培技术.pdf,3
4,杂交水稻高产高效实用栽培技术\n三、水分利用效率⋯⋯⋯⋯⋯⋯⋯⋯⋯⋯⋯⋯⋯⋯⋯⋯⋯⋯⋯72\...,./杂交水稻高产高效实用栽培技术.pdf,4
...,...,...,...
99,第六章杂父不稻厂重形成与田间诊断技术\n早稻不同品种3个地点平均产量以陆两优996和中嘉早3...,./杂交水稻高产高效实用栽培技术.pdf,99
100,杂交水稻高产高效实用栽培技术\n穗总粒数、结实率和千粒重品种间差异显著，可见在多穗的基础\n...,./杂交水稻高产高效实用栽培技术.pdf,100
101,第六章杂交水稻产量形成与田间诊断技术\n二、关键叶龄期\n有效分蘖临界叶龄期。指主茎总叶数减...,./杂交水稻高产高效实用栽培技术.pdf,101
102,杂交水稻高产高效实用栽培技术\n表6-4杂交水稻品种总叶片数和关键叶龄期(2007---20...,./杂交水稻高产高效实用栽培技术.pdf,102


In [9]:
from zhipuai import ZhipuAI
client = ZhipuAI(api_key='57d70e621ad1b471776291b62d9d0190.OLXwozfKyX1NRtcn')

In [10]:
SYSTEM_PROMPT_SUBG = """
你是一个知识图谱构建专家，用户会给你一段文本，你的任务是准确地从这段文本里提取出实体和它们之间的关系，
重点考虑与水稻生产技术相关的知识，比如高产、播种育苗、田间管理、病虫害防治、水稻减灾等等。

你可以按下面步骤进行：
1. 实体可能是人、物体、时间、地点、组织等，尽可能保证实体的原子性和多样性。
2. 一个实体可能与多个实体有关系，尽可能多地提取出关系。
3. 将提取出的实体和关系按照如下三元组格式输出，只需要输出结果，不需要解释思路。
   足球|属于|球类运动
   人|踢|足球
   疫苗|预防|疾病
"""
def subg(text: str):
  res = client.chat.completions.create(
    model="glm-4-air",
    messages=[
      {"role": "system", "content": SYSTEM_PROMPT_SUBG},
      {"role": "user", "content": text},
    ]
  )
  return res.choices[0].message

In [11]:
from tqdm import tqdm

with open("edges-0-149.2.csv", "w+") as f:
  for i, row in tqdm(df_chunks.iterrows()):
    if 0 <= i < 150:
      content = subg(row["text"]).content
      lines = content.split("\n")
      for line in lines:
        f.write(f"{line}|{i}\n")
      f.flush()

104it [17:57, 10.36s/it]


In [12]:
with open('edges.csv', 'w') as w:
    with open('edges-0-149.2.csv') as r:
        for line in r:
            cells = line.split("|")
            if len(cells) != 4:
                continue
            w.write(line)

In [13]:
df_all = pd.read_csv("edges.csv", sep="|", names=["u", "e", "v", "d"]).dropna()
df_all

,u,e,v,d
0,杂交水稻,属于,农作物,0
1,高产,描述,杂交水稻,0
2,高效实用栽培技术,应用于,杂交水稻,0
3,邹应斌,是,主编,0
4,黄敏万克江,是,副主编,0
...,...,...,...,...
2101,黄熟期,叶片颜色,转为淡黄,103
2102,根系活力,诊断方法,看白根、褐根数量,103
2103,根系活力,诊断方法,手拔稻株,103
2104,后期施用氮肥,要谨慎,防止后期贪青和倒伏,103


In [14]:
nodes: set[str] = set()
links: dict[str, int] = dict()
for i, row in df_all.iterrows():
    u = row["u"]
    v = row["v"]
    e = row["e"]
    d = row["d"]
    nodes.add(u)
    nodes.add(v)
    k = f'{u}|{v}|{e}'
    if k not in links:
        links[k] = 0
    links[k] += 1

In [15]:
len(nodes)

2346

In [16]:
import random

data = {
    'nodes': [],
    'links': []
}
for node in list(nodes):
    data['nodes'].append({
        'id': node,
        'group': random.choice([0,1,2,3,4]),
    })
for k, w in links.items():
    u, v, e = k.split('|')
    data['links'].append({
        'source': u,
        'target': v,
        'value': w,
        'label': e,
    })
data

{'nodes': [{'id': '水分管理措施  ', 'group': 2},
  {'id': '库容', 'group': 0},
  {'id': 'P区', 'group': 1},
  {'id': '早熟早稻', 'group': 4},
  {'id': '双季杂交水稻', 'group': 1},
  {'id': '籽粒灌浆速率', 'group': 4},
  {'id': '翟虎渠', 'group': 4},
  {'id': '早稻陆两优996', 'group': 0},
  {'id': '壮籽肥', 'group': 4},
  {'id': '主茎总叶片数-伸长节间数', 'group': 2},
  {'id': '移栽密度', 'group': 2},
  {'id': '2002—2010年', 'group': 3},
  {'id': '热传导率', 'group': 0},
  {'id': '研究施肥技术', 'group': 0},
  {'id': '高秆水稻', 'group': 1},
  {'id': '金优253浅水灌溉翻耕水分利用率', 'group': 3},
  {'id': '传统型', 'group': 1},
  {'id': '叶片形态', 'group': 3},
  {'id': '天优华占', 'group': 3},
  {'id': '双季稻全生育期时长', 'group': 0},
  {'id': '源的能力', 'group': 0},
  {'id': '施肥条件', 'group': 4},
  {'id': '磷钾肥', 'group': 2},
  {'id': '倒3.5叶伸出', 'group': 4},
  {'id': '一致', 'group': 0},
  {'id': '83.68ab', 'group': 4},
  {'id': '栽秧后的温度和基本苗数有关', 'group': 2},
  {'id': '大穗型杂交水稻', 'group': 3},
  {'id': '生长速率模型', 'group': 1},
  {'id': '维护保养', 'group': 0},
  {'id': '温光反应特性', 'group': 3},
  {'

In [17]:
import json

with open('g.json', 'w') as w:
    json.dump(data, w, ensure_ascii=False)